## This code is used to bulkify all the data from different sources.

In [3]:
library(rhdf5)
library(dplyr)
library(Matrix)
library(mgcv)
library(data.table)

for (chr in c(1:22, "X")){
    higashi_summary = fread("/p/keles/schic/volumeA/Duan2020/duan2020_summary.txt")
    cells = paste("cell_", 0:9229, sep = "")
    agg = c()
    for (k in 1:length(unique(higashi_summary$cell_type))){
        values = rep(0, length(h5read(paste0("/p/keles/schic/volumeA/Duan2020/higashi11132021/temp/chr", chr, "_duan_embed_nbr_3_impute.hdf5"), cells[1])))
        for (i in 1:9230){
            if (higashi_summary$cell_type[i] == unique(higashi_summary$cell_type)[k]){
                values = values + h5read(paste0("/p/keles/schic/volumeA/Duan2020/higashi11132021/temp/chr", chr, "_duan_embed_nbr_3_impute.hdf5"), cells[i])
            }
        }
        temp = cbind(t(h5read(paste0("/p/keles/schic/volumeA/Duan2020/higashi11132021/temp/chr", chr, "_duan_embed_nbr_3_impute.hdf5"), "coordinates")),
              values)
        temp = cbind(temp, unique(higashi_summary$cell_type)[k])
        temp = data.frame(temp)
        temp = temp %>% mutate_at(c("V1", "V2", "values"), as.numeric) %>% 
        rename(binA = V1, binB = V2, count = values, cluster = V4) %>% 
        mutate(binA = binA * 1000000, binB = binB * 1000000) %>% mutate(agg = count)
        agg = rbind(agg, temp)
        agg = agg %>% group_by(binA, binB, cluster) %>% summarise(agg = sum(agg))
    }
    saveRDS(agg, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_Kim2020chr", chr, ".rds"))
}


`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'binA', 'binB'. You can override using the `.groups` argument.
`summarise

In [4]:
for (chr in c(1:22, "X")){
    agg = readRDS(paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_Kim2020chr", chr, ".rds"))
    options(scipen = 9)
    agg$chrA = paste0("chr", chr)
    agg$chrB = paste0("chr", chr)
    clusters = c("GM12878", "H1Esc", "HAP1", "HFF", "IMR90")
    for (i in 1:length(clusters)){
        aggs = agg[agg$cluster == clusters[i], ]
        aggs = data.frame(0, aggs$chrA, aggs$binA, 0, 1, aggs$chrB, aggs$binB, 1, aggs$agg)
        write.table(aggs, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_", 
                                        clusters[i], "chr", chr, ".bed"), row.names = FALSE,
                    col.names = FALSE, quote = FALSE, sep = "\t")
    }
}

In [5]:
for (chr in c(1:22, "X")){
    clusters2 = paste0(clusters, "chr", chr)
    for(i in 1:length(clusters)){
      convert_bam_to_wig <- sprintf("java -jar /u/s/s/sshen82/Rfile/juicerdir/scripts/common/juicer_tools.jar pre -r 1000000 /p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_%s.bed /p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_%s.hic /u/s/s/sshen82/Rfile/juicerdir/references/hg19.chrom.sizes",
                                    clusters2[i], clusters2[i])
      #print(convert_bam_to_wig)
      system(convert_bam_to_wig)
    }
}

In [11]:
library(data.table)
library(Matrix)
library(stringr)
clusters = c("GM12878", "H1Esc", "HAP1", "HFF", "IMR90")
for (j in c(1:22, "X")){
    agg = c()
    for (i in 1:length(clusters)){
        hic_mat = as.matrix(fread(paste0("/p/keles/schic/volumeA/Duan2020/schicluster1mbBulk/", clusters[i], "_chr", j, ".txt")))
        hic_mat[lower.tri(hic_mat)] = 0
        output = as(hic_mat, "dgTMatrix")
        output = data.frame(binA = output@i * 1000000, binB = output@j * 1000000, 
                            cluster = clusters[i], agg = output@x) 
        agg = rbind(agg, output)
    }
    saveRDS(agg, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_Kim2020chr", j, ".rds"))
}


In [12]:
for (chr in c(1:22, "X")){
    agg = readRDS(paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_Kim2020chr", chr, ".rds"))
    options(scipen = 9)
    agg$chrA = paste0("chr", chr)
    agg$chrB = paste0("chr", chr)
    clusters = c("GM12878", "H1Esc", "HAP1", "HFF", "IMR90")
    for (i in 1:length(clusters)){
        aggs = agg[agg$cluster == clusters[i], ]
        aggs = data.frame(0, aggs$chrA, aggs$binA, 0, 1, aggs$chrB, aggs$binB, 1, aggs$agg)
        write.table(aggs, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_", 
                                        clusters[i], "chr", chr, ".bed"), row.names = FALSE,
                    col.names = FALSE, quote = FALSE, sep = "\t")
    }
}

In [13]:
for (chr in c(1:22, "X")){
    clusters2 = paste0(clusters, "chr", chr)
    for(i in 1:length(clusters)){
      convert_bam_to_wig <- sprintf("java -jar /u/s/s/sshen82/Rfile/juicerdir/scripts/common/juicer_tools.jar pre -r 1000000 /p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_%s.bed /p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_%s.hic /u/s/s/sshen82/Rfile/juicerdir/references/hg19.chrom.sizes",
                                    clusters2[i], clusters2[i])
      #print(convert_bam_to_wig)
      system(convert_bam_to_wig)
    }
}

In [ ]:
library(rhdf5)
library(dplyr)
library(Matrix)
library(mgcv)
library(data.table)

for (chr in c(1:22, "X")){
    higashi_summary = fread("/p/keles/schic/volumeA/Ecker2019/human/Ecker2019_summary.txt")
    cells = paste("cell_", 0:4237, sep = "")
    agg = c()
    for (k in 1:length(unique(higashi_summary$cell_type))){
        values = rep(0, length(h5read(paste0("/p/keles/schic/volumeD/Ecker2019/ProcessedData/higashi11132021/temp/chr", chr, "_ecker_embed_nbr_5_impute.hdf5"), cells[1])))
        for (i in 1:4238){
            if (higashi_summary$cell_type[i] == unique(higashi_summary$cell_type)[k]){
                values = values + h5read(paste0("/p/keles/schic/volumeD/Ecker2019/ProcessedData/higashi11132021/temp/chr", chr, "_ecker_embed_nbr_5_impute.hdf5"), cells[i])
            }
        }
        temp = cbind(t(h5read(paste0("/p/keles/schic/volumeD/Ecker2019/ProcessedData/higashi11132021/temp/chr", chr, "_ecker_embed_nbr_5_impute.hdf5"), "coordinates")),
              values)
        temp = cbind(temp, unique(higashi_summary$cell_type)[k])
        temp = data.frame(temp)
        temp = temp %>% mutate_at(c("V1", "V2", "values"), as.numeric) %>% 
        rename(binA = V1, binB = V2, count = values, cluster = V4) %>% 
        mutate(binA = binA * 1000000, binB = binB * 1000000) %>% mutate(agg = count)
        agg = rbind(agg, temp)
        agg = agg %>% group_by(binA, binB, cluster) %>% summarise(agg = sum(agg))
    }
    saveRDS(agg, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_Lee2019chr", chr, ".rds"))
}

library(data.table)
library(Matrix)
library(stringr)
clusters = c("Astro", "Endo", "L23", "L4", "L5", "L6", "MG", "MP", "Ndnf", "ODC", "OPC", "Pvalb", "Sst", "Vip")
for (j in c(1:22, "X")){
    agg = c()
    for (i in 1:length(clusters)){
        hic_mat = as.matrix(fread(paste0("/p/keles/schic/volumeA/Ecker2019/human/schicluster1mbBulk/", clusters[i], "_chr", j, ".txt")))
        hic_mat[lower.tri(hic_mat)] = 0
        output = as(hic_mat, "dgTMatrix")
        output = data.frame(binA = output@i * 1000000, binB = output@j * 1000000, 
                            cluster = clusters[i], agg = output@x) 
        agg = rbind(agg, output)
    }
    saveRDS(agg, file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_Lee2019chr", j, ".rds"))
}

In [10]:
clusters = c("GM12878", "H1Esc", "HAP1", "HFF", "IMR90")
for (i in 1:length(clusters)){
    tempCluster = c()
    for (chr in c(1:22, "X")){
            write.table(fread(paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/higashi_", 
            clusters[i], "chr", chr, ".bed"))[, c(2, 3, 6, 7, 9)], 
                        file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/binpair/higashi_", 
                                clusters[i], "_chr", chr, ".binPair"), row.names = FALSE,
            col.names = FALSE, quote = FALSE, sep = "\t")
    }
}

for (i in 1:length(clusters)){
    for (chr in c(1:22, "X")){
        write.table(fread(paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/pseudoBulk/schicluster_", 
                                        clusters[i], "chr", chr, ".bed"))[, c(2, 3, 6, 7, 9)], 
                    file = paste0("/p/keles/schic/volumeC/SiqiShen/code/BandNormPaper/binpair/schicluster_", 
                                        clusters[i], "_chr", chr, ".binPair"), row.names = FALSE,
                    col.names = FALSE, quote = FALSE, sep = "\t")
    }
}